In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances    #-}

### Syntax

In [2]:
data Register = ACC
              | NIL
              | IN
              | OUT
              | Port String
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Reg Register
             | Con Int
             deriving (Read)

type Condition = Int -> Bool
type Label = String

data Operator = MOV Register
              | SWP
           -- | SAV -> MOV ACC BAK
           -- | NOP -> ADD NIL
              | ADD
              | SUB
              | NEG
              | Jmp Condition Label     -- for parsing only
              | JRO Condition
           -- | JMP | JEZ | JNZ | JGZ | JLZ
              | HCF
              deriving (Show, Read)

data Operation = O Operator Operand
               deriving (Show, Read)


-- instances

instance Show Operand where
  show (Reg x) = show x
  show (Con x) = show x

instance Show Condition where
  show c = case map c [(-1)..1] of
             [True, True, True ] -> "__"
             [False,True, False] -> "EZ"
             [True, False,True ] -> "NZ"
             [False,False,True ] -> "GZ"
             [True, False,False] -> "LZ"

instance Read Condition where
  readsPrec _ _ = []

In [3]:
import qualified Data.Map as Map

type State = Register -> Int

data Node = Node [Operation] State

instance Show Node where
  show (Node os s) = show os

type Program = Map.Map String Node

### Parsing

In [4]:
import Text.Read
import Data.Maybe
import Control.Monad
import Data.Either

import Parsing

In [5]:
goodchar :: Parser Char
goodchar = sat (`notElem` " :#\n")

word :: Parser String
word = token $ many goodchar

name :: Parser String
name = token $ do char '@'
                  word

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)
       <|> Port <$> name

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do symbol "MOV"
                o <- operand
                r <- register
                return $ O (MOV r) o
         <|> do symbol "SAV"
                return $ O (MOV BAK) (Reg ACC)
         <|> do symbol "NOP" 
                return $ O ADD (Reg NIL)
         <|> do symbol "ADD"
                O ADD <$> operand
         <|> do symbol "SUB"
                O SUB <$> operand
         <|> do symbol "JMP"
                w <- word
                return $ O (Jmp (const True) w) (Reg NIL)
         <|> do symbol "JEZ"
                w <- word
                return $ O (Jmp (==0)        w) (Reg NIL)
         <|> do symbol "JNZ"
                w <- word
                return $ O (Jmp (/=0)        w) (Reg NIL)
         <|> do symbol "JGZ"
                w <- word
                return $ O (Jmp (>0)         w) (Reg NIL)
         <|> do symbol "JLZ"
                w <- word
                return $ O (Jmp (<0)         w) (Reg NIL)
         <|> do symbol "JRO"
                O (JRO (const True)) <$> operand
         <|> do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operator)
                return $ O (read xs :: Operator) (Reg NIL)

                
label :: Parser Label
label = token $
          do w <- word
             char ':'
             return w

node :: Parser [Either Operation Label]
node = many $
        Left  <$> operation
    <|> Right <$> label

process :: [Either Operation Label] -> (Label -> Int) -> [Operation] -> [Operation]
process [] f os = p os
  where p [] = []
        p xs = p (init xs) ++ [q (last xs)]
          where q (O (Jmp c s) _) = O (JRO c) (Con (f s - length (init xs)))
                q x         = x
process (Left  o : es) f n = process es f (n ++ [o])
process (Right l : es) f n = process es f' n
  where f' l' | l' == l   = if all isRight es then 0 else length n
              | otherwise = f l'

parseNode :: String -> [Operation]
parseNode xs = case parse node xs of
                 [(p , [])] -> process p (const undefined) []
                 [(_ , s)]  -> error ("syntax: unparsed string " ++ s)
                 _          -> error "syntax: failed to parse"
                 
makeNode :: String -> Node
makeNode xs = Node (parseNode xs) (const 0)

In [6]:
namedNode :: Parser (String, [Either Operation Label])
namedNode = do x <- name
               y <- node
               return (x, y) 
           
program :: Parser [(String, [Either Operation Label])]
program = many namedNode

processProgram :: [(String, [Either Operation Label])] -> Program
processProgram = foldr (\ x -> Map.insert (fst x) (Node (process (snd x) (const undefined) []) (const 0))) Map.empty

parseProgram :: String -> Program
parseProgram = processProgram . fst . head . parse program

### Interpreting

In [7]:
clamp :: Int -> Int -> Int -> Int
clamp l h x = max l $ min h x

update :: Register -> Int -> Node -> Node
update NIL _ n = n
update r   x (Node os s) = Node os s'
  where s' r' | r' == r   = clamp (-999) 999 x
              | otherwise = s r'

get :: Register -> Node -> Int
get r (Node os s) = s r

len :: Node -> Int
len (Node os _) = length os

current :: Node -> Operation
current (Node os s) = os !! s IP

inc :: Node -> Node
inc n = update IP ((get IP n + 1) `mod` len n) n

jump :: Int -> Node -> Node
jump i n = update IP (clamp 0 (len n-1) i) n

eval :: Operand -> Node -> Int
eval (Reg r) s = get r s
eval (Con x) _ = x

exec :: Node -> Node
exec n = e' (current n) where
  e' :: Operation -> Node
  e' (O a b) = e a (eval b n) where
    e :: Operator -> Int -> Node
    e (MOV r) o = update r o $ inc n
    e  SWP    _ = update BAK (get ACC n) $ update ACC (get BAK n) $ inc n
    e  ADD    o = update ACC (get ACC n + o) $ inc n
    e  SUB    o = update ACC (get ACC n - o) $ inc n
    e  NEG    o = update ACC (negate $ get ACC n) $ inc n
    e (JRO c) o = if c $ get ACC n then jump (get IP n + o) n else inc n
    e  HCF    _ = error "halted and caught fire"

run :: Node -> Int -> Int
run n i = get OUT $ iterate exec (update IN i n) !! 1000

In [8]:
padRight :: a -> Int -> [a] -> [a]
padRight  c n xs = take n $ xs ++ repeat c

log :: Node -> Int -> Int -> [String]
log n i c = map d $ take c $ iterate exec (update IN i n)
  where d n = show (get IP n) ++ ": " ++ padRight ' ' 11 (show (current n)) ++ " | " ++ show (get ACC n) ++ " (" ++ show (get BAK n) ++ ") " ++ show (get IN n) ++ " " ++ show (get OUT n) ++ "\n"

### Testing

In [9]:
example1 = 
  unlines [
    "  MOV IN  ACC",
    "  SAV        ",
    "  MOV 1   ACC",
    "             ",
    "L:SWP        ",
    "  JEZ E      ",
    "  JLZ F      ",
    "  SUB 1      ",
    "  SWP        ",
    "  ADD ACC    ",
    "  JMP L      ",
    "             ",
    "F:HCF        ",
    "             ",
    "E:SWP        ",
    "  MOV ACC OUT"]

putStr example1

  MOV IN  ACC
  SAV        
  MOV 1   ACC
             
L:SWP        
  JEZ E      
  JLZ F      
  SUB 1      
  SWP        
  ADD ACC    
  JMP L      
             
F:HCF        
             
E:SWP        
  MOV ACC OUT

In [10]:
parsed1 = makeNode example1
parsed1

[O (MOV ACC) IN,O (MOV BAK) ACC,O (MOV ACC) 1,O SWP NIL,O (JRO EZ) 7,O (JRO LZ) 5,O SUB 1,O SWP NIL,O ADD ACC,O (JRO __) -6,O HCF NIL,O SWP NIL,O (MOV OUT) ACC]

In [11]:
map (run parsed1) [0..11]

[1,2,4,8,16,32,64,128,256,512,999,999]

In [12]:
putStr $ concat $ log parsed1 4 35

0: O (MOV ACC) | 0 (0) 4 0
1: O (MOV BAK) | 4 (0) 4 0
2: O (MOV ACC) | 4 (4) 4 0
3: O SWP NIL   | 1 (4) 4 0
4: O (JRO EZ)  | 4 (1) 4 0
5: O (JRO LZ)  | 4 (1) 4 0
6: O SUB 1     | 4 (1) 4 0
7: O SWP NIL   | 3 (1) 4 0
8: O ADD ACC   | 1 (3) 4 0
9: O (JRO __)  | 2 (3) 4 0
3: O SWP NIL   | 2 (3) 4 0
4: O (JRO EZ)  | 3 (2) 4 0
5: O (JRO LZ)  | 3 (2) 4 0
6: O SUB 1     | 3 (2) 4 0
7: O SWP NIL   | 2 (2) 4 0
8: O ADD ACC   | 2 (2) 4 0
9: O (JRO __)  | 4 (2) 4 0
3: O SWP NIL   | 4 (2) 4 0
4: O (JRO EZ)  | 2 (4) 4 0
5: O (JRO LZ)  | 2 (4) 4 0
6: O SUB 1     | 2 (4) 4 0
7: O SWP NIL   | 1 (4) 4 0
8: O ADD ACC   | 4 (1) 4 0
9: O (JRO __)  | 8 (1) 4 0
3: O SWP NIL   | 8 (1) 4 0
4: O (JRO EZ)  | 1 (8) 4 0
5: O (JRO LZ)  | 1 (8) 4 0
6: O SUB 1     | 1 (8) 4 0
7: O SWP NIL   | 0 (8) 4 0
8: O ADD ACC   | 8 (0) 4 0
9: O (JRO __)  | 16 (0) 4 0
3: O SWP NIL   | 16 (0) 4 0
4: O (JRO EZ)  | 0 (16) 4 0
11: O SWP NIL   | 0 (16) 4 0
12: O (MOV OUT) | 16 (0) 4 0

In [13]:
run parsed1 (-1)

: 

In [14]:
example2 =
  unlines [
    "  MOV 20 ACC ",
    "  JRO IN     ",
    "  JMP E      ",
    "             ",
    "2:SUB 10     ",
    "  JMP E      ",
    "             ",
    "4:SUB 1      ",
    "  JMP E      ",
    "             ",
    "6:ADD 1      ",
    "  JMP E      ",
    "             ",
    "8:ADD 10     ",
    "             ",
    "E:MOV ACC OUT"]
           
putStr example2

  MOV 20 ACC 
  JRO IN     
  JMP E      
             
2:SUB 10     
  JMP E      
             
4:SUB 1      
  JMP E      
             
6:ADD 1      
  JMP E      
             
8:ADD 10     
             
E:MOV ACC OUT

In [15]:
parsed2 = makeNode example2
parsed2

[O (MOV ACC) 20,O (JRO __) IN,O (JRO __) 8,O SUB 10,O (JRO __) 6,O SUB 1,O (JRO __) 4,O ADD 1,O (JRO __) 2,O ADD 10,O (MOV OUT) ACC]

In [16]:
map (run parsed2) [1..11]

[20,10,20,19,20,21,20,30,20,20,20]

In [17]:
biggie = unlines
  ["@1            ",
   "   MOV 0 ACC  ",
   "   SUB IN     ",
   "&: ADD @2     ",
   "   MOV -2 @2  ",
   "   JLZ &      ",
   "   MOV @2 @2  ",
   "   MOV ACC @2 ",
   "              ",
   "@2            ",
   "&: MOV ACC @1 ",
   "   SUB @1     ",
   "   JGZ &      ",
   "   MOV @1 OUT "]
   
parseProgram biggie

fromList [("1",[O (MOV ACC) 0,O SUB IN,O ADD Port "2",O (MOV (Port "2")) -2,O (JRO LZ) -2,O (MOV (Port "2")) Port "2",O (MOV (Port "2")) ACC]),("2",[O (MOV (Port "1")) ACC,O SUB Port "1",O (JRO GZ) -2,O (MOV OUT) Port "1"])]